Importamos librerias

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Cargamos datasets

In [27]:
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')

In [28]:
# Verificación inicial
print(f"Sell-In: {sellin.shape[0]} filas y {sellin.shape[1]} columnas")
print(f"Productos: {productos.shape[0]} filas y {productos.shape[1]} columnas")
print(f"Stocks: {stocks.shape[0]} filas y {stocks.shape[1]} columnas")

Sell-In: 2945818 filas y 7 columnas
Productos: 1262 filas y 6 columnas
Stocks: 13691 filas y 3 columnas


In [29]:
# 3. MERGE INICIAL
df = sellin.merge(productos, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(f"Ventas-Productos-Stocks: {df.shape[0]} filas y {df.shape[1]} columnas")

Ventas-Productos-Stocks: 2988650 filas y 13 columnas


In [30]:
productos_clean = productos.drop_duplicates(subset=['product_id'], keep='first')
print(productos_clean.shape)

(1251, 6)


Hacemos el merge final

In [31]:
df = sellin.merge(productos_clean, on="product_id", how="left")
df = df.merge(stocks, on=["product_id", "periodo"], how="left")
print(sellin.shape)
print(df.shape)

(2945818, 7)
(2945818, 13)


# Dataset: <periodo, producto>

Tomamos con agregacion periodo y product_id

In [33]:
# Convertimos periodo a datetime
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")

# Determinar vida útil de cada producto (primer y último periodo)
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()

# Expandimos cada producto con todos los periodos de su vida útil
periodos_producto = []
for _, row in vida_producto.iterrows():
    periodos = pd.date_range(start=row["min"], end=row["max"], freq="MS")
    for p in periodos:
        periodos_producto.append((p, row["product_id"]))

df_producto_periodo = pd.DataFrame(periodos_producto, columns=["periodo_dt", "product_id"])



# Agregar columna periodo en formato AAAAMM
df_producto_periodo["periodo"] = df_producto_periodo["periodo_dt"].dt.strftime("%Y%m").astype(int)

df_producto_periodo.drop(columns=['periodo_dt'],inplace=True)
# Ordenar por periodo_dt (ascendente) y luego por product_id
df_producto_periodo = df_producto_periodo.sort_values(by=["product_id", "periodo"], ascending=True).reset_index(drop=True)



###########
toneladas_vendidas = df.copy()
# Agregar los datos por periodo y product_id para obtener la serie temporal
# Sumamos tn, cust_request_qty y cust_request_tn por periodo y product_id
toneladas_vendidas = df.groupby(['periodo', 'product_id']).agg({
    'tn': 'sum',
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum'
}).reset_index()

# Paso 5: Unir con las toneladas efectivamente vendidas (tn)
df_merge = df_producto_periodo.merge(toneladas_vendidas[["periodo", "product_id", "tn","cust_request_qty", "cust_request_tn"]],
                             on=["periodo", "product_id"],
                             how="left")

print(df_merge.shape)
df_merge["tn"] = df_merge["tn"].fillna(0)


df_merge = df_merge.merge(productos_clean[['product_id', 'cat1', 'cat2', 'cat3','brand','sku_size']], on='product_id', how='left')
print(df_merge.shape)
df_merge = df_merge.merge(stocks[['product_id', 'periodo', 'stock_final']], on=['product_id', 'periodo'], how='left')
print(df_merge.shape)



# precios cuidados
# Hacemos el merge por product_id y periodo
df_precios = df[['product_id', 'periodo_dt', 'plan_precios_cuidados']].drop_duplicates()
periodos_producto = []
for _, row in vida_producto.iterrows():
    product_id = row['product_id']
    min_fecha = row['min']
    max_fecha = row['max']
    periodos = pd.date_range(start=min_fecha, end=max_fecha, freq='MS')
    for p in periodos:
        periodos_producto.append((product_id, p))

df_periodos = pd.DataFrame(periodos_producto, columns=['product_id', 'periodo_dt'])
df_final = df_periodos.merge(df_precios, on=['product_id', 'periodo_dt'], how='left')
df_final["periodo"] = df_final["periodo_dt"].dt.strftime("%Y%m").astype(int)
df_final.drop(columns=['periodo_dt'],inplace=True)

df_merge = df_merge.merge(df_final[['product_id', 'periodo', 'plan_precios_cuidados']], on=['product_id', 'periodo'], how='left')
df_merge.shape

(31522, 5)
(31522, 10)
(31522, 11)


(31522, 12)

Feature Engineering

In [ ]:
ts = df_merge.copy()

# Convertir el periodo a formato datetime
ts['periodo_dt'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Convertir las columnas de categoría a tipo 'category' para que las detecte LGBM
ts['cat1'] = ts['cat1'].astype('category')
ts['cat2'] = ts['cat2'].astype('category')
ts['cat3'] = ts['cat3'].astype('category')
ts['brand'] = ts['brand'].astype('category')
ts['sku_size'] = ts['sku_size'].astype('category')

# Crear características adicionales
ts['crisis'] = (ts['periodo_dt'].dt.year == 2019) & (ts['periodo_dt'].dt.month == 8)
ts['quarter'] = ts['periodo_dt'].dt.quarter
ts['month'] = ts['periodo_dt'].dt.month
ts['year'] = ts['periodo_dt'].dt.year
ts['season'] = ts['periodo_dt'].apply(lambda x: 1 if x.month in [6, 7, 8] else 0)
ts['tn_diff'] = ts.groupby('product_id')['tn'].diff()
ts['rolling_mean'] = ts.groupby('product_id')['tn'].rolling(window=3).mean().reset_index(level=0, drop=True)
ts['interaction'] = ts['year'] * ts['month']

# Normalización por producto
# ts['tn_norm'] = ts.groupby('product_id')['tn'].transform(lambda x: (x - x.mean()) / x.std())

# Agregar lags a los datos
for lag in range(1, 13):
    ts[f'tn_lag_{lag}'] = ts.groupby('product_id')['tn'].shift(lag)

# Identificar el primer y último periodo de ventas para cada producto
ts['first_sale'] = ts.groupby('product_id')['periodo_dt'].transform('min')
ts['last_sale'] = ts.groupby('product_id')['periodo_dt'].transform('max')

# Calcular el tiempo desde la primera venta para cada registro
ts['months_since_launch'] = (ts['periodo_dt'] - ts['first_sale']).dt.days // 30  # en meses

# Crear una categoría de madurez basada en el tiempo desde la primera venta
conditions = [
    (ts['months_since_launch'] < 6),
    (ts['months_since_launch'] >= 6) & (ts['months_since_launch'] < 18),
    (ts['months_since_launch'] >= 18) & (ts['months_since_launch'] < 30),
    (ts['months_since_launch'] >= 30)
]
choices = ['new', 'growth', 'mature', 'decline']
ts['grado_de_madurez'] = np.select(conditions, choices, default='unknown')

# One-Hot Encode the grado_de_madurez feature
ts = pd.get_dummies(ts, columns=['grado_de_madurez'], drop_first=True)





# PESOS para el LGB: podria cambiarlo: por cantidad de ventas.
# Paso 1: Calcular la suma total por producto
participacion = ts.groupby('product_id')['tn'].sum()
# Paso 2: Calcular el total global
total_global = participacion.sum()
# Paso 3: Calcular la proporción por producto
participacion = participacion / total_global
participacion.name = 'participacion_tn'
# Paso 4: Merge con el DataFrame original
ts = ts.merge(participacion, on='product_id', how='left')







# Agrupar por categorias y ver estadisticas





# stock de prodctos: velocidad de rotacion
ts['stock_ratio'] = ts['tn'] / ts['stock_final']
ts['stock_ratio'] = ts.apply(
    lambda x: x['tn'] / x['stock_final'] if x['stock_final'] > 0 else 0,
    axis=1
)
ts['stock_ratio'] = ts['stock_ratio'].replace([np.inf, -np.inf], 0).fillna(0)




# Crecimiento del stock entre periodos
# Útil para detectar si el producto está acumulando inventario o escaseando.
ts['stock_growth'] = ts.groupby('product_id')['stock_final'].pct_change() #  calcula el cambio porcentual entre el valor actual y el valor anterior en la columna stock_final para cada grupo. 
ts['stock_growth'] = ts['stock_growth'].replace([np.inf, -np.inf], 0).fillna(0)




# Relación stock actual vs. promedio histórico
# Promedio histórico del stock por producto
avg_stock = ts.groupby('product_id')['stock_final'].transform('mean')
ts['stock_vs_avg'] = ts['stock_final'] / avg_stock
ts['stock_vs_avg'] = ts['stock_vs_avg'].replace([np.inf, -np.inf], 0).fillna(0)




# Stock lagueado (ej: t-1, t-2)
# Ideal para que LightGBM aprenda con información de meses previos.
ts['stock_lag1'] = ts.groupby('product_id')['stock_final'].shift(1)
ts['stock_lag1'] = ts['stock_lag1'].replace([np.inf, -np.inf], 0).fillna(0)
ts['stock_lag2'] = ts.groupby('product_id')['stock_final'].shift(2)
ts['stock_lag2'] = ts['stock_lag2'].replace([np.inf, -np.inf], 0).fillna(0)


# Categorizar el nivel de stock
# Podés discretizar la variable si pensás que LightGBM puede beneficiarse de eso.
ts['stock_level'] = pd.qcut(ts['stock_final'], q=4, labels=['Muy bajo', 'Bajo', 'Medio', 'Alto'])
# ts['stock_level'] = ts['stock_level'].replace([np.inf, -np.inf], 0).fillna(0)
ts['stock_level'] = ts['stock_level'].cat.codes

######## EL STOCK SE PUEDE RECONSTRUIR EN BASE A LAS TONELADAS VENDIDAS ########




# Clustering de productos




# Porcentaje vendido al top 13 de clientes: Nueva columna que, para cada combinación periodo-producto, indique qué porcentaje de toneladas fue vendido a los top 13 clientes.
# Paso 1: Identificar los top 13 clientes
df_copy = df.copy() 
top_13 = (df_copy.groupby('customer_id')['tn'].sum()
          .sort_values(ascending=False)
          .head(13)
          .index)
# Paso 2: Calcular toneladas por periodo-producto para top13
df_copy['is_top13'] = df_copy['customer_id'].isin(top_13)
agregado_total = df_copy.groupby(['periodo', 'product_id'])['tn'].sum()
agregado_top13 = df_copy[df_copy['is_top13']].groupby(['periodo', 'product_id'])['tn'].sum()
# Paso 3: Crear DataFrame de proporción
df_prop = (agregado_top13 / agregado_total).reset_index(name='porcentaje_top13')
# Paso 4: Merge con tu dataset original
ts = ts.merge(df_prop, on=['periodo', 'product_id'], how='left')



# Numero de clientes distintos por producto y mes: Esto indica cuán diversificada es la demanda por producto en cada periodo.
clientes_distintos = df.groupby(['periodo', 'product_id'])['customer_id'].nunique().reset_index(name='n_clientes')
ts = ts.merge(clientes_distintos, on=['periodo', 'product_id'], how='left')



# Concentración (índice de Herfindahl): Podés calcular el índice de concentración por producto y mes. 
# El índice de Herfindahl es la suma de los cuadrados de las participaciones de los clientes:
# Calcular la participación de cada cliente por periodo-producto
participaciones = df.groupby(['periodo', 'product_id', 'customer_id'])['tn'].sum()
# Calcular el índice de Herfindahl-Hirschman
participaciones_pct = participaciones.groupby(['periodo', 'product_id']).apply(
    lambda x: ((x / x.sum())**2).sum()
).reset_index(name='hh_index')
# Merge con tu dataframe agregado por periodo y producto
ts = ts.merge(participaciones_pct, on=['periodo', 'product_id'], how='left')





# Tasa de repetición de clientes
# % de clientes que ya compraron el producto en el período anterior
# Paso 1: Agrupar y obtener clientes únicos por período y producto
clientes_por_periodo = df.groupby(['periodo', 'product_id'])['customer_id'].unique().reset_index()
# Paso 2: Ordenar
clientes_por_periodo = clientes_por_periodo.sort_values(['product_id', 'periodo'])
# Paso 3: Shift preservando la estructura (usando transform)
clientes_por_periodo['clientes_prev'] = (
    clientes_por_periodo.groupby('product_id')['customer_id']
    .transform(lambda x: x.shift(1))
)
# Paso 4: Función corregida para tasa de repetición
def tasa_repeticion(row):
    clientes_actuales = set(row['customer_id']) if isinstance(row['customer_id'], np.ndarray) else set()
    clientes_anteriores = set(row['clientes_prev']) if isinstance(row['clientes_prev'], np.ndarray) else set()
    
    if not clientes_actuales:
        return 0.0
    
    repetidos = clientes_actuales & clientes_anteriores
    return len(repetidos) / len(clientes_actuales)
clientes_por_periodo['tasa_repeticion'] = clientes_por_periodo.apply(tasa_repeticion, axis=1)
# Resultado
resultado = clientes_por_periodo[['periodo', 'product_id', 'tasa_repeticion']]
ts = ts.merge(resultado, on=['periodo', 'product_id'], how='left')




# Clientes nuevos para ese producto
# Construir un historial de compras por cliente-producto
df_copy = df.copy()
df_copy['first_purchase'] = df_copy.groupby(['customer_id', 'product_id'])['periodo'].transform('min')
# Cliente nuevo = primera compra de ese producto en ese mes
df_copy['cliente_nuevo'] = (df_copy['periodo'] == df_copy['first_purchase']).astype(int)
# Agregar a nivel periodo-producto
clientes_nuevos = df_copy.groupby(['periodo', 'product_id'])['cliente_nuevo'].sum().reset_index()
# Merge con el DataFrame original
ts = ts.merge(clientes_nuevos, on=['periodo', 'product_id'], how='left')






# Promedio histórico de toneladas por cliente (en un mes): promedio de compra
promedio_tn_cliente = (
    df.groupby(['product_id', 'customer_id'])['tn']
    .mean()
    .reset_index()
    .groupby('product_id')['tn']
    .mean()
    .reset_index()
    .rename(columns={'tn': 'prom_tn_cliente'})
)
ts = ts.merge(promedio_tn_cliente, on='product_id', how='left')






# Varianza de las toneladas por cliente (en un mes): dispersión en tamaño de compra
var_tn_cliente = (
    df.groupby(['periodo', 'product_id'])['tn']
    .std()
    .reset_index()
    .rename(columns={'tn': 'std_tn_cliente'})
)
ts = ts.merge(var_tn_cliente, on=['periodo', 'product_id'], how='left')





# Coeficiente de Gini de participación de clientes: desigualdad en la distribución de compras
def gini(array):
    array = np.sort(np.array(array))
    n = len(array)
    if n == 0:
        return np.nan
    cumx = np.cumsum(array, dtype=float)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

gini_por_producto = (
    df.groupby(['periodo', 'product_id'])['tn']
    .apply(gini)
    .reset_index()
    .rename(columns={'tn': 'gini_clientes'})
)
ts = ts.merge(gini_por_producto, on=['periodo', 'product_id'], how='left')










# Prediccion: SARIMA, PMDARIMA, STATSFORECAST, PROPHET, DARTS

# clientes: ocasionales, esporadicos, regulares, frecuentes



# Eventos politicos


ts['periodo_dt'] = ts['periodo_dt'].dt.year * 100 + ts['periodo_dt'].dt.month
ts['first_sale'] = ts['first_sale'].dt.year * 100 + ts['first_sale'].dt.month
ts['last_sale'] = ts['last_sale'].dt.year * 100 + ts['last_sale'].dt.month




C:\Users\iparra\AppData\Local\Temp\ipykernel_24808\2903603023.py:90: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ts['stock_growth'] = ts.groupby('product_id')['stock_final'].pct_change() #  calcula el cambio porcentual entre el valor actual y el valor anterior en la columna stock_final para cada grupo.


Defino el target

In [ ]:
# Target
ts['tn_target'] = ts['tn'].shift(-2)


Entrenamiento con lgb

In [37]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from scipy.stats import uniform
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# quito 201911 y 201912
dt_kgl = ts[ts["periodo"].isin([201912])]
ts = ts.drop(ts[ts["periodo"].isin([201911,201912])].index,axis=0)

for col in ['cat1', 'cat2', 'cat3', 'brand','sku_size']:
    if col in ts.columns:
        ts[col] = ts[col].astype('category')


feature_columns = [col for col in ts.columns if col not in ['periodo_dt', 'tn_target']]


X = ts[feature_columns]
y = ts['tn_target']


lgb_reg = lgb.LGBMRegressor(random_state=12345)

# Entrenar el modelo
lgb_reg.fit(X, y)

print("Modelo LightGBM entrenado con éxito.")




[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001707 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8623
[LightGBM] [Info] Number of data points in the train set: 29652, number of used features: 41
[LightGBM] [Info] Start training from score 42.165901
Modelo LightGBM entrenado con éxito.


Armamos dataset de predicción

In [39]:
feature_columns = [col for col in ts.columns if col not in ['periodo_dt', 'tn_target']]
X_kgl = dt_kgl[feature_columns]

productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")
# Filtrar filas
productos_filtrados = productos_a_predecir['product_id'].unique()
X_kgl = X_kgl[X_kgl['product_id'].isin(productos_filtrados)]

X_kgl['product_id'].nunique()

780

Predecimos

In [41]:
y_pred = lgb_reg.predict(X_kgl)

<!-- Guardamos la prediccion -->

In [ ]:
result = pd.DataFrame({"product_id": X_kgl["product_id"],  "tn": y_pred})
result.to_csv("./kaggle/06-lgb.csv", index=False, sep=',')
result

,product_id,tn
35,20001,1284.479988
71,20002,1350.981234
107,20003,742.383889
143,20004,673.925562
179,20005,590.792058
...,...,...
31344,21263,-0.442789
31384,21265,-0.612402
31394,21266,-0.593370
31404,21267,0.034758
